In [1]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math
from web3 import exceptions
from eth_abi import encode
import web3
from web3.exceptions import ContractLogicError, TimeExhausted

In [2]:
from web3_defi_manage3 import *

In [3]:
RPC_URL = "https://eth-sepolia.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
w3=Web3_Network(RPC_URL, PRIVATE_KEY)

In [4]:
if w3.is_connected():
    print(f"Connected to {w3.chain_id} network")
    print(f'{w3.get_network_name_from_api()}')
    print(w3.address)
    print(f'{w3.network_gas_mode}')

Connected to 11155111 network
Ethereum Sepolia
0x7868EDEfC36A74eba220C75aCFB05c59a4e55822
EIP-1559


In [5]:
w3.get_position_manager_address(w3.to_checksum_address("0x1238536071E1c677A632429e3655c799b22cDA52"))
w3.get_router_address(w3.to_checksum_address('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E'))
w3.get_factory_address(w3.to_checksum_address('0x0227628f3F023bb0B980b67D528571c95c6DaC1c'))

In [6]:
w3.get_explorer_api_key("N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M")

In [7]:
w3.GET_FACTORY_ABI()

Successfully fetched ABI from: 0x0227628f3F023bb0B980b67D528571c95c6DaC1c


In [8]:
w3.GET_ROUTER_ABI()

Successfully fetched ABI from: 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E


In [9]:
w3.GET_V3_NFPM_ABI()

Successfully fetched ABI from: 0x1238536071E1c677A632429e3655c799b22cDA52


In [10]:
pm=PositionManager(w3)

In [11]:
pm.get_all_positions()

Found 0 Uniswap V3 LP NFTs for address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822
No LP NFTs found for this address.


[]

In [12]:
pool=V3Pool.from_factory(w3,'0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238','0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',100)
pool.load()
pool.update_state()

Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5


In [13]:
pool.info()

{'sqrtPriceX96': 1142332712665385798953204140036854,
 'current_tick': 191534,
 'liquidity': 436592103153847,
 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee': 100}

In [14]:
token0=Token(w3,pool.token0)
token1=Token(w3,pool.token1)
token0.load()
token1.load()
token0.balance_of()
token1.balance_of()
print(token0.token_balance)
print(token1.token_balance)
print(pool.fee)


Successfully fetched ABI from: 0xDa317C1d3E835dD5F1BE459006471aCAA1289068
Successfully fetched ABI from: 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14
42932452
153554300807140169
100


In [23]:
pm.multicall_jit_mint(token0=token0,token1=token1,fee=pool.fee,step=100)

Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5
Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5


RuntimeError: Mint simulation failed: ('execution reverted: STF', '0x08c379a0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000035354460000000000000000000000000000000000000000000000000000000000')

In [22]:
pm.mint_params

{'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee': 100,
 'tickLower': 191513,
 'tickUpper': 191555,
 'amount0Desired': 379647730,
 'amount1Desired': 83555776763717184,
 'amount0Min': 0,
 'amount1Min': 0,
 'recipient': '0x7868EDEfC36A74eba220C75aCFB05c59a4e55822',
 'deadline': 1766528713}

In [24]:
pm.jit_mint(token0=token0,token1=token1,fee=pool.fee,slippage=0.05,step=10)

Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5
Info: Swapping 67891880228934080 WETH for USDC to balance amounts.
Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5
Action: Swapping 67891880228934080 WETH for at least 306911760 USDC
Transaction sent: b8891e6c60fc9b405cbc14e030a4151390bdb4802f87e337712de093dce0bc67
Confirmed in block 9901663
Action: Preparing to mint new LP position NFT...
Details: Token0: USDC, Token1: WETH, Fee: 100, Amounts Desired: 365997463/85662420578206089, Min Amounts: 347697589/81379299549295792, Ticks: [191523, 191545]
Transaction sent: c9f243eb2924920a407fcf0bb95175716688e2d5804d0df24756233e1de7e3b1
Confirmed in block 9901664


'c9f243eb2924920a407fcf0bb95175716688e2d5804d0df24756233e1de7e3b1'

In [26]:
pm.jit_mint(token0=token0,token1=token1,fee=pool.fee,slippage=0.05,step=100)

Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5
Info: Swapping 195074965 USDC for WETH to balance amounts.
Successfully fetched ABI from: 0xFeEd501c2B21D315F04946F85fC6416B640240b5
Action: Swapping 195074965 USDC for at least 38561799731247552 WETH
Transaction sent: 942f6648e9f3f138d8217a64484ce564906d124d16787cb4274f862cb983fc1a
Confirmed in block 9901683
Action: Preparing to mint new LP position NFT...
Details: Token0: USDC, Token1: WETH, Fee: 100, Amounts Desired: 193162034/40591368138155324, Min Amounts: 183503932/38561799731247560, Ticks: [191471, 191673]
Transaction sent: 1130b3562c549574df9a72f97b9c218164f148ecd93ac6bddeb5637b4e0c73e4
Confirmed in block 9901684


'1130b3562c549574df9a72f97b9c218164f148ecd93ac6bddeb5637b4e0c73e4'